<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


Este laboratorio contiene las siguientes tareas:

* **TAREA 1:** Marcar todos los sitios de lanzamiento en un mapa
* **TAREA 2:** Marcar los lanzamientos exitosos/fallidos de cada sitio en el mapa
* **TAREA 3:** Calcular las distancias entre un sitio de lanzamiento y sus proximidades

Después de completar las tareas anteriores, debería poder encontrar algunos patrones geográficos sobre los sitios de lanzamiento.

Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

Si necesita refrescar su memoria sobre el folium, puede descargar y consultar este laboratorio de folium anterior:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [28]:
## Tarea 1: Marcar todos los sitios de lanzamiento en un mapa


Primero, intentemos agregar la ubicación de cada sitio en un mapa usando las coordenadas de latitud y longitud del sitio.


El siguiente conjunto de datos con el nombre `spacex_launch_geo.csv` es un conjunto de datos aumentado con latitud y longitud agregadas para cada sitio.


In [81]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [82]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Las coordenadas anteriores son simplemente números simples que no pueden brindarte ninguna información intuitiva sobre dónde están esos sitios de lanzamiento. Si eres muy bueno en geografía, puedes interpretar esos números directamente en tu mente. Si no, eso también está bien. Visualicemos esas ubicaciones marcándolas en un mapa.

Primero debemos crear un objeto `Mapa` en forma de folio, con una ubicación central inicial que será el Centro Espacial Johnson de la NASA en Houston, Texas.


In [20]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [29]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

y deberías encontrar un pequeño círculo amarillo cerca de la ciudad de Houston y puedes hacer zoom para ver un círculo más grande.


Ahora, agreguemos un círculo para cada sitio de lanzamiento en el marco de datos `launch_sites`


*TODO:* Crear y agregar `folium.Circle` y `folium.Marker` para cada sitio de lanzamiento en el mapa del sitio


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [49]:
# Initial the map
# Inicializar el mapa centrado en las coordenadas promedio de los sitios
nasa_coordinate = [launch_sites_df["Lat"].mean(), launch_sites["Long"].mean()]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Agregar círculos y etiquetas a cada sitio de lanzamiento
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row["Lat"], row["Long"]],
        radius=1000,  # Radio de 1000 metros
        color="#000000",  # Color del borde
        fill=True,
        fill_color="#3186cc",  # Color de relleno
        fill_opacity=0.7
    ).add_child(folium.Popup(row["Launch Site"])).add_to(site_map)
    
    # Agregar un marcador con el nombre del sitio
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        popup=row["Launch Site"],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(site_map)

# Mostrar el mapa
site_map


El mapa generado con los sitios de lanzamiento marcados debería verse similar al siguiente:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Ahora, puedes explorar el mapa acercando o alejando las áreas marcadas
e intentar responder las siguientes preguntas:

* ¿Están todos los sitios de lanzamiento cerca de la línea del Ecuador?
* ¿Están todos los sitios de lanzamiento muy cerca de la costa?

Intenta también explicar tus hallazgos.


In [58]:
# Task 2: Mark the success/failed launches for each site on the map

# Inicializar el mapa centrado en las coordenadas promedio
site_map = folium.Map(location=[spacex_df["Lat"].mean(), spacex_df["Long"].mean()], zoom_start=5)

# Función para asignar colores en función del resultado
def get_color(outcome):
    return 'green' if outcome == 'Success' else 'red'

# Agregar marcadores con colores en función del resultado
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Site: {row['Launch Site']}<br>Landing Outcome: {row['Landing Outcome']}",
        icon=folium.Icon(color=get_color(row['Landing Outcome']), icon="rocket", prefix="fa")
    ).add_to(site_map)

# Mostrar el mapa
site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [54]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


A continuación, crearemos marcadores para todos los registros de lanzamiento. Si un lanzamiento fue exitoso `(class=1)`, entonces usamos un marcador verde y si un lanzamiento falló, usamos un marcador rojo `(class=0)`


Tenga en cuenta que un lanzamiento solo se produce en uno de los cuatro sitios de lanzamiento, lo que significa que muchos registros de lanzamiento tendrán exactamente la misma coordenada. Los grupos de marcadores pueden ser una buena forma de simplificar un mapa que contiene muchos marcadores que tienen la misma coordenada.


Let's first create a `MarkerCluster` object


In [55]:
marker_cluster = MarkerCluster()


*TODO:* Crear una nueva columna en el marco de datos `spacex_df` llamada `marker_color` para almacenar los colores de los marcadores según el valor de `class`


In [86]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Crear una función que asigna colores según el valor de la columna 'class'
# Agregar círculos y etiquetas para cada sitio de lanzamiento
# Inicializar el mapa centrado en la posición promedio de los sitios de lanzamiento
site_map = folium.Map(location=[30, -100], zoom_start=4)

# Agregar círculos y etiquetas a cada sitio de lanzamiento
for site in launch_sites:
    # Agregar círculo amarillo
    folium.Circle(
        location=[site["lat"], site["long"]],
        radius=50000,  # Radio del círculo
        color="#FDB813",  # Borde amarillo
        fill=True,
        fill_color="#FDB813",  # Relleno amarillo
        fill_opacity=0.5
    ).add_to(site_map)
    
    # Agregar marcador con texto del número de lanzamientos
    folium.Marker(
        location=[site["lat"], site["long"]],
        icon=DivIcon(
            icon_size=(50, 50),  # Tamaño del ícono
            icon_anchor=(0, 0),  # Punto de anclaje
            html=f'<div style="font-size: 12px; color: #d35400;"><b>{site["count"]}</b><br>{site["site"]}</div>'
        )
    ).add_to(site_map)

# Mostrar el mapa
site_map


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [65]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


A partir de los marcadores etiquetados por color en los grupos de marcadores, debería poder identificar fácilmente qué sitios de lanzamiento tienen tasas de éxito relativamente altas.


In [90]:
# TASK 3: Calculate the distances between a launch site to its proximities
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

# Define the Haversine formula to calculate distances
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Distance in kilometers

# Coordinates of the launch site (example: CCAFS LC-40)
launch_site = {'site': 'CCAFS LC-40', 'lat': 28.562302, 'lon': -80.577356}

# Coordinates of nearby points of interest
proximities = [
    {'name': 'Railway Station', 'lat': 28.563, 'lon': -80.586},
    {'name': 'Highway Intersection', 'lat': 28.570, 'lon': -80.585},
    {'name': 'Coastline', 'lat': 28.565, 'lon': -80.570},
    {'name': 'Nearby City Center', 'lat': 28.611, 'lon': -80.807}
]

# Calculate distances and store results
results = []
for point in proximities:
    distance = haversine(launch_site['lat'], launch_site['lon'], point['lat'], point['lon'])
    results.append({'Site': launch_site['site'], 'Proximity': point['name'], 'Distance_km': round(distance, 2)})

# Convert results into a DataFrame
distances_df = pd.DataFrame(results)

# Display the results
print("Distances between the launch site and nearby proximities:")
print(distances_df)



Distances between the launch site and nearby proximities:
          Site             Proximity  Distance_km
0  CCAFS LC-40       Railway Station         0.85
1  CCAFS LC-40  Highway Intersection         1.14
2  CCAFS LC-40             Coastline         0.78
3  CCAFS LC-40    Nearby City Center        23.07


Next, we need to explore and analyze the proximities of launch sites.


Primero, agreguemos una `Posición del mouse` en el mapa para obtener las coordenadas del mouse sobre un punto del mapa. De esta manera, mientras explora el mapa, puede encontrar fácilmente las coordenadas de cualquier punto de interés (como el ferrocarril).

In [92]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Ahora acérquese a un sitio de lanzamiento y explore su proximidad para ver si puede encontrar fácilmente algún ferrocarril, autopista, costa, etc. Mueva el mouse a estos puntos y marque sus coordenadas (mostradas en la parte superior izquierda) para indicar la distancia al sitio de lanzamiento.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [93]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Marque un punto en la costa más cercana usando MousePosition y calcule la distancia entre el punto de la costa y el sitio de lanzamiento.

In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
